In [ ]:
from utils.radar_capture_utils import AWR6843, DCA1000, async_radar_saver
#from utils.camera_capture_utils import camera, async_video_saver
from utils.imu_capture_utils import realsense_imu
from utils.realsense_capture_utils import realsense_camera, async_video_saver

from utils.test_utils import test_radar, test_camera, test_imu

import time
import os
import numpy as np
from tqdm import tqdm
from scipy.signal import stft
import matplotlib.pyplot as plt

print(time.strftime('%Y%h%d-%H%M'))

In [ ]:
output_dat_path='./Test_data/'+time.strftime('%Y_%h_%d')
if not os.path.exists(output_dat_path):
    os.mkdir(output_dat_path)
    print('path_created!')
else:
    print('path already exists!')
    

# Step1: Initialise radars and camera

Init Radar:

In [ ]:
ports=['COM5','COM6']

In [ ]:
config_file_name=b"./configs/xwr68xx_profile-test.cfg"
config_file_name=b"./config68.cfg"

radar=AWR6843(radar_name="Radar1",
             cmd_serial_port=ports[0],
             dat_serial_port=ports[1],
             config_file_name=config_file_name)
radar.stop_radar()

Init DCA:

In [ ]:
dca = DCA1000()

dca.startup(timeout=0.3)
time.sleep(1)

Init Camera:

In [ ]:
camera_width=640
camera_height=480
camera_fps=30
cam=realsense_camera(
    camera_name='Camera1',
    res_w=camera_width,
    res_h=camera_height,
    fps=camera_fps)
cam.start()

Init IMU:

In [ ]:
imu=realsense_imu()
imu.start()

# Step2: test all sensors
Common failures: <br>
1- Radars not initializing : check the serial ports from device manager <br>
2- Camera failing: unplug camera and replug <br>
3- Address not available: unplug DCA, restart kernel <br>
4- Timeout error: unplug dca, restart kernel <br>
5- serial port busy or unavailable: unplug the radar and restart kernel <br>
6- empty data packets: restart kernel, if didnt work, restart dca and radar  <br>


In [ ]:
valid_criteria={
    'min_doppler_threshold':0.25,
    'max_doppler_threshold':3,
    'min_range_threshold':5,
    'max_range_threshold':20,
    'min_points_per_radar_frame':1,
    'min_radar_frames':0,
    'min_camera_frames':10
}

In [ ]:
camera_ready=test_camera(cam,capture_len=5)


In [ ]:
camera_depth=np.uint8(np.repeat(np.expand_dims(np.stack(cam.depth,0),-1),3,-1))
camera_frame = np.stack(cam.frames,0)[...]
print(camera_frame.shape)
plt.imshow(np.uint8(np.mean(camera_depth,0)))

In [ ]:
print(np.stack(cam.depth,0).shape)
depth_frame=np.stack(cam.depth,0)
plt.imshow(np.uint8(depth_frame[10,:,:]))
plt.figure()
plt.imshow(np.uint8(depth_frame[50,:,:]))


In [ ]:
imu_ready=test_imu(imu,capture_len=5)

In [ ]:
radar_ready=test_radar(dca,radar,config_file_name,capture_len=5)
radar.stop_radar()


if radar_ready and camera_ready:
    print('-----\nAll sensors ready!')
else:
    print('-----\nInit failed!')    

# Step3: Capture!

In [ ]:
#How many episodes to capture ? set either the number of minutes or number of episodes (max number will be chosen)
capture_notes="swipe_right Soheil" # notes for post processing: "$class $subject_name" 
capture_len_minutes = 0 # total capture time in minutes
total_num_captures = 1    # total number of captured episodes

capture_period=10       # time per episode (should match waveform)
inter_capture_delay=0.1 # delay between captures
testing_period=5        # test every nth capture
controlled_capture=False # controlled or uncontrolled capture

##############################
total_interesting_snapshots=0
f_list=[]
loop_time=capture_period+inter_capture_delay

if total_num_captures==0:
    total_num_captures=np.int32(np.ceil(60*capture_len_minutes/loop_time))

out_folder=os.path.join(output_dat_path,time.strftime('%Y%h%d-%H%M')+'/')
if not os.path.exists(out_folder):
    os.mkdir(out_folder)
    os.mkdir(out_folder+'/radar/')
    os.mkdir(out_folder+'/camera/')
    os.mkdir(out_folder+'/depth/')
print(out_folder)

with open(out_folder+'readme.txt','w') as f:
    f.write(capture_notes)


if radar_ready and camera_ready:
    
    print('3')
    time.sleep(1)
    print('2')
    time.sleep(1)
    print('1')
    time.sleep(1)
    
    t0=time.time()
    for i in range(total_num_captures):
        radar.stop_radar()

        dca.data=[]
        camera_frame=[]
        
        time.sleep(0.01)

        radar.start_radar()    

#        cam.start(capture_period)
        cam.start_capture_timed(record_time=loop_time)
        
        time.sleep(loop_time)
        
        print('\n--------\nSnapshot '+str(i)+' out of '+str(total_num_captures))
                
        t1=time.time()
        print('Measured loop time:'+str(t1-t0)+' seconds')
        t0=t1
        
        data_buffer=bytearray(np.concatenate(dca.data.copy()))
        print(f'{radar.radar_name} data size: {len(data_buffer)}')
               
        radar_data_list=[data_buffer]
        camera_frame=np.stack(cam.frames,0)[...,::-1]
        camera_depth=np.uint8(np.repeat(np.expand_dims(np.stack(cam.depth,0),-1),3,-1))        
#        camera_frame=np.stack([frame[0] for frame in cam.data],0)
#        camera_depth=np.stack([frame[1] for frame in cam.data],0)        
#        if len(zed_cam.data)>0:
#            camera_frame=np.stack([zed_cam.data[0],zed_cam.data[len(zed_cam.data)//2],zed_cam.data[-1]],0)
#        if not i%testing_period:
#            valid_flags = test_snapshot(radar_data_list=radar_data_list,
#                                        camera=zed_cam,
#                                        valid_criteria=valid_criteria)
#            print(valid_flags)
            
        if controlled_capture: ######### save

            fname=time.strftime('%Y%m%d%H%M%S')
            
            #### save camera in background
            f_list.append(async_video_saver(res_w=camera_width,
                                            res_h=camera_height,
                                            fps=camera_fps,
                                            data=camera_frame,
                                            path_out=os.path.join(out_folder,'camera',fname+'.avi')))
            #### save camera in background
            f_list.append(async_video_saver(res_w=camera_width,
                                            res_h=camera_height,
                                            fps=camera_fps,
                                            data=camera_depth,
                                            path_out=os.path.join(out_folder,'depth',fname+'.avi')))
            
            #### save radar in background
            f_list.append(async_radar_saver(radar_data=radar_data_list,
                                            camera_frame=[camera_frame[[0,camera_frame.shape[0]//2,-1],...]],#camera_frame,
                                            camera_depth=[],#cam.depth,#np.stack(cam.depth,0),#camera_depth,                                           
                                            path_out=os.path.join(out_folder,'radar',fname+'.npz')))
            
            total_interesting_snapshots+=1
else:
    print('Error!\n Sensors not ready!')

print('\n-----\n'+out_folder)


# Summerize last episode:

In [ ]:
plt.figure()
camera_frame=np.stack(cam.frames,0)[...,::-1]
plt.imshow(np.uint8(np.mean(camera_frame,0)))
plt.figure()
depth_frame=np.stack(cam.depth,0)
plt.imshow(np.uint8(np.mean(depth_frame,0)))


In [ ]:
import pandas as pd
p=DCA1000.decode_data(None,bytearray(np.concatenate(dca.data.copy())))

pcloud=pd.concat(p)
plt.plot(pcloud['frame'],pcloud['x'],'-o',label='x')
plt.plot(pcloud['frame'],pcloud['y'],'-o',label='y')
plt.plot(pcloud['frame'],pcloud['z'],'-o',label='z')
plt.xlabel('Frame')
plt.ylabel('Location(meters)')
plt.legend()
plt.figure()
plt.plot(pcloud['frame'],pcloud['v'],'-o',label='v')
plt.xlabel('Frame')
plt.ylabel('Velocity(m/s)')


# Capture notes:
Use the cell bellow as a notepad for documentation (to be cleaned up before each capture)